### Student Information
Name: 黃粲育

Student ID: 110164511

GitHub ID: gardra

Kaggle name: O口O (Gardra)

Kaggle private scoreboard snapshot:

[Snapshot](img/pic0.png)

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM2022-Lab2-master Repo](https://github.com/keziatamus/DM2022-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm2022-isa5810-lab2-homework) regarding Emotion Recognition on Twitter by this link https://www.kaggle.com/t/2b0d14a829f340bc88d2660dc602d4bd. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   
    Submit your last submission __BEFORE the deadline (Nov. 22th 11:59 pm, Tuesday)_. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 25th 11:59 pm, Friday)__. 

#　Step 1,  preprocessing 

    In this step, I find that tweet is the most important features, so I get them from tweets_DM.json. Because I want to use transformer to do this homework, so I use AutoTokenizer from transformers libary. The tokenizer can auto tokenize the sentence and then convert into input format of the model. 

    The tokenizer of "vinai/bertweet-base" model has one paramter "normalization", it will nomalize the string, ex: convert emotion to sting, convert @someone to @USER .....　Because I think it can make the performance of the model Better, I set it to True. 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
sample_sub = pd.read_csv("/kaggle/input/dm2022-isa5810-lab2-homework/sampleSubmission.csv")

data_ident = pd.read_csv("/kaggle/input/dm2022-isa5810-lab2-homework/data_identification.csv")
label = pd.read_csv("/kaggle/input/dm2022-isa5810-lab2-homework/emotion.csv")
import json
tweets = []
with open("/kaggle/input/dm2022-isa5810-lab2-homework/tweets_DM.json", encoding='utf-8') as f :
    for line in f.readlines():
        tweets.append(json.loads(line))

/kaggle/input/encodeofdata/train_enc.pickle
/kaggle/input/encodeofdata/pred_enc.pickle
/kaggle/input/dm2022-isa5810-lab2-homework/tweets_DM.json
/kaggle/input/dm2022-isa5810-lab2-homework/sampleSubmission.csv
/kaggle/input/dm2022-isa5810-lab2-homework/data_identification.csv
/kaggle/input/dm2022-isa5810-lab2-homework/emotion.csv


In [2]:
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()

    wandb.login(key="79a90f31e7b3e3989e6eefdf648c598f1c1d762a")
    anony = None
except:
    anony = "must"
    print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
datas = {"tweet_id":[], "text":[]}

In [4]:
for i in tweets:
    datas['tweet_id'].append(i['_source']['tweet']['tweet_id'])
    datas['text'].append(i['_source']['tweet']['text'])
    

In [5]:
datas = pd.DataFrame(datas)

In [6]:
datas

,tweet_id,text
0,0x376b20,"People who post ""add me on #Snapchat"" must be ..."
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #..."
2,0x28b412,"Confident of your obedience, I write to you, k..."
3,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,0x2de201,"""Trust is not the same as faith. A friend is s..."
...,...,...
1867530,0x316b80,When you buy the last 2 tickets remaining for ...
1867531,0x29d0cb,I swear all this hard work gone pay off one da...
1867532,0x2a6a4f,@Parcel2Go no card left when I wasn't in so I ...
1867533,0x24faed,"Ah, corporate life, where you can date <LH> us..."


In [7]:
all_data = datas.merge(data_ident, how="left", on='tweet_id')

all_data = all_data.merge(label, how="left", on='tweet_id')

In [8]:
import emoji

from sklearn.model_selection import train_test_split

import torch 

In [9]:

from transformers import AutoModel, AutoTokenizer


In [10]:
MODEL ="vinai/bertweet-base"

In [11]:
MODEL ="vinai/bertweet-base"

In [12]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer


In [13]:
tokenizer = AutoTokenizer.from_pretrained(MODEL, truncation=True, normalization=True)


Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
all_data.text[1]

'@brianklaas As we see, Trump is dangerous to #freepress around the world. What a <LH> <LH> #TrumpLegacy.  #CNN'

In [15]:
pred_x = all_data.loc[all_data.identification=='test']
train_x = all_data.loc[all_data.identification=='train']
train_y = train_x.emotion

In [16]:
from sklearn.preprocessing import LabelEncoder

In [17]:
labels = ['anticipation', 'sadness', 'fear', 'joy', 'anger', 'trust',
       'disgust', 'surprise']

In [18]:
le = LabelEncoder()

In [19]:
le.fit(labels)

LabelEncoder()

In [20]:
train_y = le.transform(train_y)

In [21]:
train_enc = tokenizer(train_x.text.to_list())

Token indices sequence length is longer than the specified maximum sequence length for this model (130 > 128). Running this sequence through the model will result in indexing errors


In [22]:
pred_enc = tokenizer(pred_x.text.to_list())

In [23]:
import pickle

In [26]:
tokenizer('AAAA')

{'input_ids': [0, 22292, 2], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

In [2]:
#truncate the features 

In [27]:
for i in range(len(pred_enc['input_ids'])):
    if(len(pred_enc['input_ids'][i])>128):
        pred_enc['input_ids'][i] = pred_enc['input_ids'][i][:128]
        pred_enc['token_type_ids'][i] = pred_enc['token_type_ids'][i][:128]
        pred_enc['attention_mask'][i] = pred_enc['attention_mask'][i][:128]
    

In [28]:
for i in range(len(train_enc['input_ids'])):
    if(len(train_enc['input_ids'][i])>128):
        train_enc['input_ids'][i] = train_enc['input_ids'][i][:128]
        train_enc['token_type_ids'][i] = train_enc['token_type_ids'][i][:128]
        train_enc['attention_mask'][i] = train_enc['attention_mask'][i][:128]
       

1
1
1
1
1
1
1
1
1
1
1


In [29]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Step 2,　Choose and Fine Tune the model 

    In this step, I find there are a lot of different transformer models. They use different dataset to train the model. Finally I find bertweet model. It use the tweet to train the model. The performace of bertweet is better than other bert models, so I choose it and then fine tune it. 
        
    According to resarch of the bert, the best epoch of fine tune is 2 ~ 4 , learning rates from 2e-5 ot 5e-5, batch sizes  are 16, 32 , 64, 128. I choose the epoch to 4, learning rate to 2e-5, batch size to 16. Then, fine tune the model.
        
    Beause the data is too large and I don't have too much compute reources, I skip the validation step to reduce the time of using GPU.  


In [31]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=8).to('cuda')

Downloading:   0%|          | 0.00/517M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: 

In [32]:
args = TrainingArguments(
    f"{MODEL}-finetuned",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",

    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    #load_best_model_at_end=True,
    #metric_for_best_model="accuracy",
  
)

In [33]:
train_dataset = Dataset(train_enc, train_y)
val_dataset = Dataset(tokenizer(train_x.text.to_list()[:200]), train_y[:200])

In [34]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
  
)

In [35]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1455563
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 363892
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: gardra (berttweet). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.840600,0.749964
2,0.752700,0.540169
3,0.634800,0.390438
4,0.559300,0.324302


***** Running Evaluation *****
  Num examples = 200
  Batch size = 16
Saving model checkpoint to vinai/bertweet-base-finetuned/checkpoint-90973
Configuration saved in vinai/bertweet-base-finetuned/checkpoint-90973/config.json
Model weights saved in vinai/bertweet-base-finetuned/checkpoint-90973/pytorch_model.bin
tokenizer config file saved in vinai/bertweet-base-finetuned/checkpoint-90973/tokenizer_config.json
Special tokens file saved in vinai/bertweet-base-finetuned/checkpoint-90973/special_tokens_map.json
added tokens file saved in vinai/bertweet-base-finetuned/checkpoint-90973/added_tokens.json
***** Running Evaluation *****
  Num examples = 200
  Batch size = 16
Saving model checkpoint to vinai/bertweet-base-finetuned/checkpoint-181946
Configuration saved in vinai/bertweet-base-finetuned/checkpoint-181946/config.json
Model weights saved in vinai/bertweet-base-finetuned/checkpoint-181946/pytorch_model.bin
tokenizer config file saved in vinai/bertweet-base-finetuned/checkpoint-18194

TrainOutput(global_step=363892, training_loss=0.7271096184465634, metrics={'train_runtime': 39956.0065, 'train_samples_per_second': 145.717, 'train_steps_per_second': 9.107, 'total_flos': 1.2878074025363563e+17, 'train_loss': 0.7271096184465634, 'epoch': 4.0})

In [36]:
#　test dataset 

In [37]:
# pred_enc['input_ids'].reverse()
# pred_enc['token_type_ids'].reverse()
# pred_enc['attention_mask'].reverse()

In [38]:
test_dataset = Dataset(pred_enc, [0]*len(pred_x))

# Step 3, predict the result 

In [39]:
results = trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 411972
  Batch size = 16


In [40]:
results = list(np.argmax(results.predictions, axis=-1))

In [41]:
results = le.inverse_transform(results)

In [42]:
res = {"id":pred_x.tweet_id.to_list(), "emotion":results.tolist()}

In [43]:
res = pd.DataFrame(res)

In [44]:
res.to_csv("submission.csv", index=False)